# Tutorial: Benzene and Biphenyl

## Introduction
In this tutorial, we'll focus on generating integrals for the PPP model considering real organic molecules: Benzene and Biphenyl 

To do this, we are going to consider the most general occupation-number Hamiltonian we consider is the generalized Pariser-Parr-Pople (PPP) Hamiltonian:

$$\hat{H}_{\text{PPP}} = \sum_{pq} h_{pq} a_p^\dagger a_q + \sum_p U_p \hat{n}_{p\alpha}\hat{n}_{p\beta} + \frac{1}{2}\sum_{p\ne q} \gamma_{pq} (\hat{n}_{p \alpha} + \hat{n}_{p \beta} - Q_p)(\hat{n}_{q \alpha} + \hat{n}_{q \beta} - Q_q) $$


## Example: Defining PPP Hamiltonian

In order to define a PPP hamiltonina, it's necessary to provide the parameter `gamma`. Performing a Full Configuration Iteraction of the hamiltonian, varying $\beta$


In [29]:
import sys 
sys.path.insert(0, '../')
import numpy as np 
import sys 
import numpy as np 
from moha import HamPPP 
from pyscf import gto, scf, fci 

RXY = np.array([0.00504725, 0.09930802, 0.16008021])
 
# Define the benzene system with a 6-membered ring structure 
system = [('C1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'C1', 1)] 


#Define U_onsite, alpha and gamma matrix 
u_onsite = np.array([10.84, 10.84, 10.84, 10.84, 10.84, 10.84])/2
alpha = 0 
 
# Define the gamma matrix here (example shape) 
norb = 6  # Number of orbitals for benzene (6 carbons) 
gamma_matrix = np.zeros((norb, norb))

gamma_a_a1 = 5.27760 
gamma_a_a2 = 3.86206 
gamma_a_a3 = 3.48785 
 
# Fill the matrix according to the given rules 
for a in range(norb): 
    gamma_matrix[a-1, a] = gamma_a_a1 
    gamma_matrix[a-2, a] = gamma_a_a2 
    gamma_matrix[a-3, a] = gamma_a_a3 
     
    gamma_matrix[a, a-1] = gamma_a_a1 
    gamma_matrix[a, a-2] = gamma_a_a2 
    gamma_matrix[a, a-3] = gamma_a_a3

    gamma_matrix[a, a] = 10.84
    
gamma_lib = gamma_matrix - np.diag(np.diag(gamma_matrix))
# Loop to vary beta from 0 to -5
for beta in np.linspace(0, -5, num=6):
    # Generate the PPP object
    ppp_hamiltonian = HamPPP(system, alpha=alpha, beta=beta, u_onsite=u_onsite, gamma=gamma_lib, charges=np.array([1 for _ in range(6)]))
    
    # Generate the 0, 1, 2 body integral elements
    e01 = ppp_hamiltonian.generate_zero_body_integral() 
    h11 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
    h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis',sym = 4) 
    print(e01)
 
    # Convert h2 to chemists notation 
    h21_ch = 1*np.transpose(h2, (0, 2, 1, 3)) 
    norb = h11.shape[0] 
    nelec = norb  # Assuming one electron per site  

    # Perform the FCI calculation 
    e_fci, ci = fci.direct_spin1.kernel(1*h11, 2*h21_ch, norb, nelec, ecore=e01, nroots=1)
    print(f'Beta: {beta}, FCI Energy: {e_fci}')


65.30151
Beta: 0.0, FCI Energy: -1.4210854715202004e-14
65.30151
Beta: -1.0, FCI Energy: -2.779514577625193
65.30151
Beta: -2.0, FCI Energy: -9.084603708594514
65.30151
Beta: -3.0, FCI Energy: -16.484889549995387
65.30151
Beta: -4.0, FCI Energy: -24.19501406479904
65.30151
Beta: -5.0, FCI Energy: -32.024692914537425


# PPP Benzene: Building gamma


MoHa package offers a functionality to compute `gamma` using the module `PariserParr` and the function `compute_gamma`. 

In order to do that, user can provide all the paramters of the function or let the library compute with the default values:

* If user does not provide `U_xy`(potential energies for sites) or `R_xy`matrix with distances of the sites; Those values are going to be computed using the distances provided by `connectivity`, using the formulas below, based on the deafult values of `affinity_dct` and `atom_dict`:

$$ U_x = \alpha_x - A_x $$

$$
\overline{U}_{XY} = \frac{1}{2}(U_X + U_Y)
$$

User can also provide this dictionaries(affinity_dct` and `atom_dict`) to the function and proceed in the same way.


Finally, `gamma` is computed using the formula:

$$
\gamma_{XY} = \frac{\overline{U}_{XY}}{\overline{U}_{XY} R_{XY} + e^{-\frac{1}{2} \overline{U}_{XY}^2 R_{XY}^2}}
$$







Defining the Benzene system following th same sttructure defined in the previous tutorials, we decide to move on with Full Configuration Iteraction of the hamiltonian, defining  `gamma`

Varying $\beta$ from 0 to -5, in this piece of code we prove that we can achieve the spectrum of energies reported in Bendazzoli et al.

In [30]:
import sys 
sys.path.insert(0, '../')
import numpy as np 
import sys 
import numpy as np 
from moha import HamPPP 
from pyscf import gto, scf, fci 

RXY = np.array([0.00504725, 0.09930802, 0.16008021])
 
# Define the benzene system with a 6-membered ring structure 
system = [('C1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'C1', 1)] 


#Define U_onsite, alpha and gamma matrix 
u_onsite = np.array([10.84, 10.84, 10.84, 10.84, 10.84, 10.84])/2
alpha = 0 
 
# Define the gamma matrix here (example shape) 
norb = 6  # Number of orbitals for benzene (6 carbons) 
gamma_matrix = np.zeros((norb, norb))

gamma_a_a1 = 5.27760 
gamma_a_a2 = 3.86206 
gamma_a_a3 = 3.48785 
 
# Fill the matrix according to the given rules 
for a in range(norb): 
    gamma_matrix[a-1, a] = gamma_a_a1 
    gamma_matrix[a-2, a] = gamma_a_a2 
    gamma_matrix[a-3, a] = gamma_a_a3 
     
    gamma_matrix[a, a-1] = gamma_a_a1 
    gamma_matrix[a, a-2] = gamma_a_a2 
    gamma_matrix[a, a-3] = gamma_a_a3

    gamma_matrix[a, a] = 10.84
    
gamma_lib = gamma_matrix - np.diag(np.diag(gamma_matrix))
# Loop to vary beta from 0 to -5
for beta in np.linspace(0, -5, num=6):
    # Generate the PPP object
    ppp_hamiltonian = HamPPP(system, alpha=alpha, beta=beta, u_onsite=u_onsite, gamma=gamma_lib, charges=np.array([1 for _ in range(6)]))
    
    # Generate the 0, 1, 2 body integral elements
    e01 = ppp_hamiltonian.generate_zero_body_integral() 
    h11 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
    h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis',sym = 4) 
    print(e01)
 
    # Convert h2 to chemists notation 
    h21_ch = 1*np.transpose(h2, (0, 2, 1, 3)) 
    norb = h11.shape[0] 
    nelec = norb  # Assuming one electron per site  

    # Perform the FCI calculation 
    e_fci, ci = fci.direct_spin1.kernel(1*h11, 2*h21_ch, norb, nelec, ecore=e01, nroots=1)
    print(f'Beta: {beta}, FCI Energy: {e_fci}')


65.30151
Beta: 0.0, FCI Energy: -1.4210854715202004e-14
65.30151
Beta: -1.0, FCI Energy: -2.779514577625193
65.30151
Beta: -2.0, FCI Energy: -9.084603708594514
65.30151
Beta: -3.0, FCI Energy: -16.484889549995387
65.30151
Beta: -4.0, FCI Energy: -24.19501406479904
65.30151
Beta: -5.0, FCI Energy: -32.024692914537425


## Pariser Parr Atomic dictionary

The previous approach requires to build the $\gamma$ matrix directly, but MoHa package supports a method of computing the matrix based on the distance($R_{XY}$) beetween the atoms and the potential($U_{XY}$). 


If the value of `gamma` is not provided, it is computed using the formula:

$$
\gamma_{XY} = \frac{\overline{U}_{XY}}{\overline{U}_{XY} R_{XY} + e^{-\frac{1}{2} \overline{U}_{XY}^2 R_{XY}^2}}
$$



In [31]:
import sys
import numpy as np
from moha import HamPPP
from moha.rauk import PariserParr
from pyscf import gto, scf, fci

# Define the benzene system with a 6-membered ring structure

system = [('C1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'C1', 1)] 

# Define the distances
RXY = [0.00504725, 0.09930802, 0.16008021]

# Build the adjacency matrix directly
Rxy_matrix = np.array([
    [0.0,        RXY[0],    RXY[1],    RXY[2],    RXY[1],    RXY[0]],
    [RXY[0],    0.0,        RXY[0],    RXY[1],    RXY[2],    RXY[1]],
    [RXY[1],    RXY[0],    0.0,        RXY[0],    RXY[1],    RXY[2]],
    [RXY[2],    RXY[1],    RXY[0],    0.0,        RXY[0],    RXY[1]],
    [RXY[1],    RXY[2],    RXY[1],    RXY[0],    0.0,        RXY[0]],
    [RXY[0],    RXY[1],    RXY[2],    RXY[1],    RXY[0],    0.0]
])

u_onsite = np.array([10.84, 10.84, 10.84, 10.84, 10.84, 10.84]) / 2
gamma_matrix = PariserParr.compute_gamma(system, U_xy=u_onsite, Rxy_matrix=Rxy_matrix)

alpha = 0

# Loop to vary beta from 0 to -5
for beta in np.linspace(0, -5, num=6):
    ppp_hamiltonian = HamPPP(system, alpha=alpha, gamma = gamma_matrix, beta=beta, u_onsite=u_onsite, charges=np.array([1 for _ in range(6)]))
    
    # Generate the 0, 1, 2 body integral elements
    e0 = ppp_hamiltonian.generate_zero_body_integral() 
    h1 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
    h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis', sym=4)
    
    # Convert h2 to chemists notation
    h2_ch = np.transpose(h2, (0, 2, 1, 3))
    norb = h1.shape[0]
    nelec = norb  # Assuming one electron per site  

    # Perform the FCI calculation
    e_fci, ci = fci.direct_spin1.kernel(1*h1, 2*h2_ch, norb, nelec, ecore=e0, nroots=1)
    print(f'Beta: {beta}, FCI Energy: {e_fci}')

Beta: 0.0, FCI Energy: 7.105427357601002e-15
Beta: -1.0, FCI Energy: -2.950976576402695
Beta: -2.0, FCI Energy: -9.425258557059209
Beta: -3.0, FCI Energy: -16.75428207213497
Beta: -4.0, FCI Energy: -24.36705165013518
Beta: -5.0, FCI Energy: -32.1115278385854


## Heteroatoms

The ModelHamiotonian package also supports the possibility of using different type of atoms, using the Rauk’s table (RAUK, 2001).

To do this, simply replace an atom of C with one of B.


In [32]:
import sys 
import numpy as np 
from moha import HamPPP 
from pyscf import gto, scf, fci 
 
# Define the benzene system with a 6-membered ring structure 
system = [('B1', 'C2', 1), ('C2', 'C3', 1),  
          ('C3', 'C4', 1), ('C4', 'C5', 1),  
          ('C5', 'C6', 1), ('C6', 'B1', 1)] 
 
u_onsite = np.array([10.84, 10.84, 10.84, 10.84, 10.84, 10.84]) / 2

gamma_matrix = PariserParr.compute_gamma(system, U_xy=u_onsite)
 
norb = 6  # Number of orbitals for benzene (6 carbons) 
ppp_hamiltonian = HamPPP(system, u_onsite=u_onsite, gamma=gamma_matrix, charges=np.array([1 for _ in range(6)]))

e0 = ppp_hamiltonian.generate_zero_body_integral() 
h1 = ppp_hamiltonian.generate_one_body_integral(dense=True, basis='spatial basis') 
h2 = ppp_hamiltonian.generate_two_body_integral(dense=True, basis='spatial basis', sym=4) 
 
# Convert h2 to chemists notation 
h2_ch = np.transpose(h2, (0, 2, 1, 3)) 
norb = h1.shape[0] 
nelec = norb  # Assuming one electron per site  

# Perform the FCI calculation 
e_fci, ci = fci.direct_spin1.kernel(h1, h2_ch, norb, nelec, ecore=e0, nroots=1)
print(f'Alpha: {ppp_hamiltonian.alpha}, Beta: {ppp_hamiltonian.beta}, FCI Energy: {e_fci}')


Alpha: -0.414, Beta: -0.0533, FCI Energy: -5.468422237553814


# References

Bendazzoli, G. L., Evangelisti, S., & Gagliardi, L. (1994). Full configuration interaction study of the ground state of closed-shell cyclic PPP polyenes. International Journal of Quantum Chemistry, 51(1), 117-123. https://doi.org/10.1002/qua.560510104



